### 6. Masked Fill 함수
- 텐서 내의 원하는 부분만 특정 값으로 채움
- 원하는 위치는 마스크로 설정

In [2]:
import torch
x = torch.FloatTensor([i for i in range(3**2)]).reshape(3, -1)
x

tensor([[0., 1., 2.],
        [3., 4., 5.],
        [6., 7., 8.]])

In [4]:
mask = x > 4
mask

tensor([[False, False, False],
        [False, False,  True],
        [ True,  True,  True]])

In [5]:
x.masked_fill(mask, value=100)

tensor([[  0.,   1.,   2.],
        [  3.,   4., 100.],
        [100., 100., 100.]])

#### 참고
- expand()에서 확장되는 차원의 데이터는 원본 데이터의 참조일 뿐, 복사가 아님
- 그러므로 값을 변경하면 함께 변경됨

### 4.2 행렬곱

In [6]:
import torch
x = torch.FloatTensor([[1, 2],
                       [3, 4],
                       [5, 6]])

y = torch.FloatTensor([[1, 2],
                       [1, 2]])
print(x.size(), y.size())

torch.Size([3, 2]) torch.Size([2, 2])


In [7]:
z = torch.matmul(x, y)
print(z.size())

torch.Size([3, 2])


#### 배치 행렬곱
- bmm(): 3차원 행렬 계산에 용이
- 3x3x2인 텐서는 3x2 크기의 행렬이 3개 있는 것으로 볼 수 있음
- 3x2x3인 텐서는 2x3 크기의 행렬 3개임
- 3x2 행렬과 2x3 행렬의 곱 3개를 병렬로 한번에 계산 -> 3x3x3 (3x3 크기의 행렬 3개)

In [8]:
x = torch.FloatTensor(3,3,2)
y = torch.FloatTensor(3,2,3)
z = torch.bmm(x,y)
print(z.size())

torch.Size([3, 3, 3])


### 4.4 선형 계층
- 3 x 2 크기의 행렬 W와 2개의 요소를 갖는 벡터 b를 선언
- 입력: 3개의 뉴련, 출력: 2개의 뉴런

In [14]:
W = torch.FloatTensor([[1, 2],
                       [3, 4],
                       [5, 6]])

b = torch.FloatTensor([2, 2])

In [18]:
# 선형 계층 함수
def linear(x, W, b):
    y = torch.matmul(x, W) + b
    return y

In [11]:
# x: 3개의 요소를 갖는 4개의 샘플 (3개의 뉴련에 샘플 4개를 연속으로 입력 -> 한번에 처리)
x = torch.FloatTensor(4,3)
x

tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])

In [19]:
# 4개의 샘플에 대한 결과값
y = linear(x, W, b)
print(y.size())
torch.Size([4,2])

torch.Size([4, 2])


torch.Size([4, 2])

In [20]:
y

tensor([[2., 2.],
        [2., 2.],
        [2., 2.],
        [2., 2.]])

#### torch.nn.Mocule 클래스 상속 받기
- torch.nn: 신경망 패키지
- Module: 신경망 구현을 위한 추상 클래스
- Module 클래스를 상속받으면 2개의 메서드를 오버라이드 해야함
    - __init__(): 계층에 필요한 변수 선언
    - forward(): 계층을 통과하는데 필요한 계산 수행(순방향 전파)

In [21]:
import torch.nn as nn

In [25]:
class MyLinear(nn.Module):

    def __init__(self, input_dim=3, output_dim=2):
        self.input_dim = input_dim
        self.output_dim = output_dim

        super().__init__()

        self.W = torch.FloatTensor(input_dim, output_dim)
        self.b = torch.FloatTensor(output_dim)

    def forward(self, x):
        # |x| = (batch_size, input_dim)
        y = torch.matmul(x, self.W) + self.b
        # |y| = (batch_size, input_dim) * (input_dim, output_dim)
        #     = (batch_size, output_dim)
        
        return y

In [31]:
linear = MyLinear(3,2)
y = linear(x)
# forward() 함수를 따로 호출하지 않고 객체명에 바로 괄호를 열어 텐서 x를 인수로 넘겨줌
# __call() 함수와 forward() 함수가 매핑되어 있음 (자동 호출)

In [30]:
y

tensor([[0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.]])

#### 올바른 방법: nn.Parameter 활용하기
- 텐서를 만들어서 nn.Parameter로 넘겨줌
- pyTorch에서 학습할 때 전달되는 파라미터 클래스

In [33]:
class MyLinear(nn.Module):
    def __init__(self, input_dim=3, output_dim=2):
        
        self.input_dim = input_dim
        self.output_dim = output_dim
        
        super().__init__()

        # 해당 부분만 수정
        self.W = nn.Parameter(torch.FloatTensor(input_dim, output_dim))
        self.b = nn.Parameter(torch.FloatTensor(output_dim))

    def forward(self, x):
        # |x| = (batch_size, input_dim)
        y = torch.matmul(x, self.W) + self.b

        return y

In [38]:
for p in linear.parameters():
    print(p)

Parameter containing:
tensor([[ 0.4854,  0.3172, -0.4132],
        [ 0.3038,  0.5400, -0.5668]], requires_grad=True)
Parameter containing:
tensor([0.1387, 0.4931], requires_grad=True)


#### 미리 정의된 선형 계층 nn.Linear

In [36]:
linear = nn.Linear(3,2)
y = linear(x)

In [37]:
for p in linear.parameters():
    print(p)

Parameter containing:
tensor([[ 0.4854,  0.3172, -0.4132],
        [ 0.3038,  0.5400, -0.5668]], requires_grad=True)
Parameter containing:
tensor([0.1387, 0.4931], requires_grad=True)


In [39]:
class MyLinear(nn.Module):
    def __init__(self, input_dim=3, output_dim=2):
        
        self.input_dim = input_dim
        self.output_dim = output_dim
        
        super().__init__()

        self.lenear = nn.Linear(inpur_dim, out_dim)

    def forward(self, x):
        # |x| = (batch_size, input_dim)
        y = torch.matmul(x, self.W) + self.b

        return y

### 5.2 MSE Loss

In [41]:
def mse(x_hat, x):
    # |x_hat| = (batch_size, dim)
    # |x|     = (batch_size, dim)
    y = ((x - x_hat)**2).mean()
    return y

In [42]:
import torch
x = torch.FloatTensor([[1, 1],
                       [2, 2]])

x_hat = torch.FloatTensor([[0, 0],
                           [0, 0]])

print(mse(x_hat, x))

tensor(2.5000)


#### torch.nn.functional 사용

In [45]:
import torch.nn.functional as F
F.mse_loss(x_hat, x)

tensor(2.5000)

In [47]:
F.mse_loss(x_hat, x, reduction='sum')

tensor(10.)

In [48]:
F.mse_loss(x_hat, x, reduction='none')

tensor([[1., 1.],
        [4., 4.]])

#### torch.nn 사용

In [49]:
import torch.nn as nn

In [51]:
mse_loss = nn.MSELoss() # mm.Module의 계층(Layer)으로 다룰 수 있음
mse_loss(x_hat, x)

tensor(2.5000)

#### 교차 엔트로피

In [52]:
F.cross_entropy(x_hat, x)

tensor(2.0794)